In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


In [2]:
# B = build_adjacency_matrix(labeled_pdb_dir='glycosciencedb/data_reformulate/')
# B.calculate_all_matrix()

In [3]:
num_test = 240 # 299 * 80% 

In [4]:
# Create = create_graph(data_dir='glycosciencedb/data_reformulate/', 
#                       atom_embed_dir='merged_embed/atom_name_embed.csv',
#                       mono_embed_dir='merged_embed/monosaccharide_embed.csv',
#                       ab_embed_dir='merged_embed/ab_embed.csv',
#                       dl_embed_dir='merged_embed/dl_embed.csv',
#                       pf_embed_dir='merged_embed/pf_embed.csv',
#                       num_test=num_test, seed=97211, use_mono=1, use_ab=1, use_dl=1, use_pf=1)

# g, test_index = Create.create_all_graph()

In [5]:
# save_graphs('glycosciencedb/graph_data/data.bin', g)

g = load_graphs('glycosciencedb/graph_data/data.bin')[0][0]

In [6]:
num_epoch = 5000
lr = 1e-3

atom_dim = 256
mono_dim = 128
ab_dim = 64
dl_dim = 64
pf_dim = 64

In [7]:
in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim
# in_size = atom_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)
features = g.ndata["feat"]
labels = g.ndata["shift_value"]
# masks = g.ndata['train_mask'], g.ndata['test_mask']


# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']
# masks = g.ndata['train_mask'], g.ndata['test_mask']

print(features.dtype)
print(labels.dtype)
# model = NMR_GCN(in_size=576, hid_size=[256, 128, 64, 32], out_size=1).to(device)
model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)
# model training

# NMR_prediction = NMR_prediction(results_dir='experimental_data_combined/graph_combined/experimental_results_all_hydrogen.csv',
#                                 model_dir='experimental_data_combined/graph_combined/Model_hydrogen_experiment_training.pt',
#                                num_epoch = num_epoch, 
#                                lr = lr)


NMR_prediction = NMR_prediction(results_dir='transfer_learning_results/2D_glycoscience/no_pretrain/training_carbon.csv',
                                results_dir_test = 'transfer_learning_results/2D_glycoscience/no_pretrain/testing_carbon.csv',
                                model_dir='transfer_learning_results/2D_glycoscience/no_pretrain/Model_Godess_carbon.pt', 
                               num_epoch = num_epoch, 
                               lr = lr)


print("Training...")
NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

# acc = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, print_out=True)
acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))


# acc1 = NMR_prediction.evaluate(g, features, labels, masksc[0], saved_model, save_train=True, save_test = False)

# acc2 = NMR_prediction.evaluate(g, features, labels, masksc[1], saved_model, save_train=False, save_test = True)


# print("train RMSE {:.4f}".format(acc1))
# print("test RMSE {:.4f}".format(acc2))

torch.float32
torch.float32
Training...


  0%|                                         | 10/5000 [00:00<05:04, 16.39it/s]

5279
1231
Epoch 00000 | Loss 6117.4482 | train_RMSE 77.1647 | test_RMSE 77.4268 
5279
1231
Epoch 00001 | Loss 5954.2803 | train_RMSE 75.7121 | test_RMSE 75.9951 
5279
1231
Epoch 00002 | Loss 5733.6377 | train_RMSE 73.5407 | test_RMSE 73.8541 
5279
1231
Epoch 00003 | Loss 5406.0547 | train_RMSE 70.3826 | test_RMSE 70.7411 
5279
1231
Epoch 00004 | Loss 4953.1870 | train_RMSE 65.9107 | test_RMSE 66.3299 
5279
1231
Epoch 00005 | Loss 4351.9438 | train_RMSE 59.7430 | test_RMSE 60.2360 
5279
1231
Epoch 00006 | Loss 3570.6541 | train_RMSE 51.4960 | test_RMSE 52.0608 
5279
1231
Epoch 00007 | Loss 2656.8545 | train_RMSE 41.0853 | test_RMSE 41.6525 
5279
1231
Epoch 00008 | Loss 1702.1713 | train_RMSE 30.2037 | test_RMSE 30.3943 
5279
1231
Epoch 00009 | Loss 914.2922 | train_RMSE 27.8935 | test_RMSE 26.6858 
5279
1231
Epoch 00010 | Loss 802.3515 | train_RMSE 38.7668 | test_RMSE 36.5813 
5279
1231
Epoch 00011 | Loss 1539.6879 | train_RMSE 42.8646 | test_RMSE 40.5988 
5279
1231
Epoch 00012 | Loss 1

  1%|▏                                        | 30/5000 [00:01<01:48, 45.65it/s]

5279
1231
Epoch 00017 | Loss 662.9049 | train_RMSE 28.1556 | test_RMSE 28.0209 
5279
1231
Epoch 00018 | Loss 795.4902 | train_RMSE 29.9326 | test_RMSE 29.8316 
5279
1231
Epoch 00019 | Loss 899.2157 | train_RMSE 30.5688 | test_RMSE 30.4461 
5279
1231
Epoch 00020 | Loss 951.4677 | train_RMSE 30.0338 | test_RMSE 29.8554 
5279
1231
Epoch 00021 | Loss 911.5764 | train_RMSE 28.4791 | test_RMSE 28.2148 
5279
1231
Epoch 00022 | Loss 817.3800 | train_RMSE 26.1645 | test_RMSE 25.7799 
5279
1231
Epoch 00023 | Loss 690.7173 | train_RMSE 23.5495 | test_RMSE 23.0052 
5279
1231
Epoch 00024 | Loss 565.6411 | train_RMSE 21.3502 | test_RMSE 20.6179 
5279
1231
Epoch 00025 | Loss 465.5881 | train_RMSE 20.4355 | test_RMSE 19.5428 
5279
1231
Epoch 00026 | Loss 427.3578 | train_RMSE 21.1015 | test_RMSE 20.1414 
5279
1231
Epoch 00027 | Loss 470.4424 | train_RMSE 22.5198 | test_RMSE 21.5697 
5279
1231
Epoch 00028 | Loss 516.5127 | train_RMSE 23.4748 | test_RMSE 22.5574 
5279
1231
Epoch 00029 | Loss 574.0748 | 

...

1231
Epoch 04947 | Loss 17.4898 | train_RMSE 2.5307 | test_RMSE 4.7170 
5279
1231
Epoch 04948 | Loss 15.6376 | train_RMSE 2.6452 | test_RMSE 4.8090 
5279
1231
Epoch 04949 | Loss 17.0011 | train_RMSE 2.8174 | test_RMSE 4.8935 
5279
1231
Epoch 04950 | Loss 16.7972 | train_RMSE 2.4466 | test_RMSE 4.7152 
5279
1231
Epoch 04951 | Loss 14.8158 | train_RMSE 2.7816 | test_RMSE 4.8931 
5279
1231
Epoch 04952 | Loss 19.5161 | train_RMSE 3.0464 | test_RMSE 5.0799 
5279
1231
Epoch 04953 | Loss 20.1703 | train_RMSE 2.5232 | test_RMSE 4.7510 
5279
1231
Epoch 04954 | Loss 17.2401 | train_RMSE 3.1339 | test_RMSE 5.0890 
5279
1231
Epoch 04955 | Loss 20.8318 | train_RMSE 3.0367 | test_RMSE 5.0892 
5279
1231
Epoch 04956 | Loss 18.7178 | train_RMSE 2.5723 | test_RMSE 4.8362 
5279
1231
Epoch 04957 | Loss 15.8814 | train_RMSE 2.8222 | test_RMSE 4.9846 
5279
1231
Epoch 04958 | Loss 17.3904 | train_RMSE 2.3910 | test_RMSE 4.7541 
5279
1231
Epoch 04959 | Loss 15.3263 | train_RMSE 2.9077 | test_RMSE 5.0239 
5279

100%|██████████████████████████████████████▊| 4983/5000 [00:53<00:00, 95.30it/s]

5279
1231
Epoch 04967 | Loss 16.2552 | train_RMSE 3.2120 | test_RMSE 5.1340 
5279
1231
Epoch 04968 | Loss 21.7980 | train_RMSE 2.8757 | test_RMSE 4.9418 
5279
1231
Epoch 04969 | Loss 18.4572 | train_RMSE 2.6060 | test_RMSE 4.7568 
5279
1231
Epoch 04970 | Loss 16.7951 | train_RMSE 3.1344 | test_RMSE 5.0666 
5279
1231
Epoch 04971 | Loss 20.0364 | train_RMSE 2.7308 | test_RMSE 4.8111 
5279
1231
Epoch 04972 | Loss 17.5023 | train_RMSE 2.6212 | test_RMSE 4.7657 
5279
1231
Epoch 04973 | Loss 16.0960 | train_RMSE 2.5559 | test_RMSE 4.7455 
5279
1231
Epoch 04974 | Loss 16.0151 | train_RMSE 2.3529 | test_RMSE 4.6564 
5279
1231
Epoch 04975 | Loss 15.7464 | train_RMSE 2.6951 | test_RMSE 4.8825 
5279
1231
Epoch 04976 | Loss 16.7104 | train_RMSE 2.5183 | test_RMSE 4.7645 
5279
1231
Epoch 04977 | Loss 15.7152 | train_RMSE 2.5729 | test_RMSE 4.8174 
5279
1231
Epoch 04978 | Loss 16.4819 | train_RMSE 2.6494 | test_RMSE 4.9039 
5279
1231
Epoch 04979 | Loss 15.8329 | train_RMSE 2.3665 | test_RMSE 4.7344 

100%|███████████████████████████████████████| 5000/5000 [00:54<00:00, 92.32it/s]


1231
Epoch 04986 | Loss 15.2874 | train_RMSE 2.4423 | test_RMSE 4.7472 
5279
1231
Epoch 04987 | Loss 18.6188 | train_RMSE 2.5668 | test_RMSE 4.8160 
5279
1231
Epoch 04988 | Loss 18.6190 | train_RMSE 2.6207 | test_RMSE 4.8473 
5279
1231
Epoch 04989 | Loss 18.0153 | train_RMSE 2.5911 | test_RMSE 4.8050 
5279
1231
Epoch 04990 | Loss 15.2921 | train_RMSE 2.6598 | test_RMSE 4.7718 
5279
1231
Epoch 04991 | Loss 15.4354 | train_RMSE 2.7052 | test_RMSE 4.7809 
5279
1231
Epoch 04992 | Loss 16.6875 | train_RMSE 2.6892 | test_RMSE 4.8196 
5279
1231
Epoch 04993 | Loss 17.2816 | train_RMSE 2.5029 | test_RMSE 4.7469 
5279
1231
Epoch 04994 | Loss 15.9787 | train_RMSE 2.4172 | test_RMSE 4.7077 
5279
1231
Epoch 04995 | Loss 15.0697 | train_RMSE 2.4011 | test_RMSE 4.7140 
5279
1231
Epoch 04996 | Loss 16.7173 | train_RMSE 2.5985 | test_RMSE 4.8074 
5279
1231
Epoch 04997 | Loss 18.0973 | train_RMSE 2.4650 | test_RMSE 4.7644 
5279
1231
Epoch 04998 | Loss 15.3803 | train_RMSE 2.5418 | test_RMSE 4.8253 
5279

In [11]:
# num_epoch = 5000
no_fine_tune = np.array([4.6210, 4.7313, 4.867, 4.4850, 4.9071])
np.mean(no_fine_tune), np.std(no_fine_tune)

(4.72228, 0.15604044860227728)

In [10]:
# num_epoch = 1000
no_fine_tune = np.array([4.8463, 4.7466, 4.7363, 4.8890, 4.8950])
np.mean(no_fine_tune), np.std(no_fine_tune)

(4.82264, 0.06846326314162947)